In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3'

In [ ]:
%run -p ../train_cnf_disentangle_rl_semisup.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/infocnf_cond_cifar10_bs900_sratio_0_5_drop_0_5_rl_stdscale_6_nsup4k_beta_0_01_run1 --seed 1 --lr 0.001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --scale_fac 1.0 --scale_std 6.0 --labeled_batch_size 450 --num_train_sup 4000 --beta 0.01
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl_semisup.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import torch.utils.data as data
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

from PIL import Image
import os.path
import errno
import codecs

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWr

Load semi-sup datasets for CIFAR10


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateI(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfu

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0001 | Time 207.6949, Epoch Time 1806.1392(1806.1392), Bit/dim 7.0173(best: inf), Bit/dim Marginal 7.0170(best: inf), Xent 2.0832, Loss 8.0589, Error 0.7076(best: inf)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0110 | Time 12.8200(13.9130) | Bit/dim 6.9789(7.3713) | Xent 2.0808(2.0931) | Loss 18.1512(19.3601) | Error 0.6956(0.7077) Steps 556(521.98) | Grad Norm 8.7336(6.0049) | Total Time 0.00(0.00)
Iter 0120 | Time 13.6994(13.8070) | Bit/dim 6.9533(7.2658) | Xent 2.0297(2.0841) | Loss 17.7922(18.9672) | Error 0.6867(0.7049) Steps 532(524.99) | Grad Norm 3.5182(6.0689) | Total Time 0.00(0.00)
Iter 0130 | Time 14.3371(13.7649) | Bit/dim 6.9220(7.1794) | Xent 2.0018(2.0728) | Loss 17.7593(18.6669) | Error 0.6711(0.7004) Steps 544(530.26) | Grad Norm 4.4385(6.6066) | Total Time 0.00(0.00)
Iter 0140 | Time 13.4474(13.7859) | Bit/dim 6.8954(7.1065) | Xent 1.9783(2.0604) | Loss 17.7006(18.4324) | Error 0.6378(0.6931) Steps 544(533.60) | Grad Norm 4.1641(6.7684) | Total

...

Epoch 0003 | Time 81.7215, Epoch Time 1460.8422(1788.2700), Bit/dim 5.5548(best: 6.0291), Bit/dim Marginal 5.5321(best: 6.0225), Xent 1.9729, Loss 6.5412, Error 0.6744(best: 0.7076)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0310 | Time 12.6698(12.9972) | Bit/dim 5.5222(5.6440) | Xent 1.8106(1.8824) | Loss 14.5371(15.5951) | Error 0.5800(0.6215) Steps 514(533.95) | Grad Norm 27.1325(26.6753) | Total Time 0.00(0.00)
Iter 0320 | Time 12.7119(12.9764) | Bit/dim 5.4857(5.6148) | Xent 1.8952(1.8910) | Loss 14.5493(15.3597) | Error 0.6556(0.6344) Steps 562(536.55) | Grad Norm 42.6124(37.0075) | Total Time 0.00(0.00)
Iter 0330 | Time 13.2585(12.9751) | Bit/dim 5.4967(5.5809) | Xent 1.8198(1.8797) | Loss 14.6849(15.1432) | Error 0.6200(0.6289) Steps 562(536.41) | Grad Norm 7.2102(33.5513) | Total Time 0.00(0.00)
Iter 0340 | Time 14.3818(13.0303) | Bit/dim 5.4520(5.5496) | Xent 1.7120(1.8565) | Loss 14.4272(14.9577) | Error 0.5622(0.6168) Steps 544(536.13) | Grad Norm 14.5590(2

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0004 | Time 99.4834, Epoch Time 1517.6105(1780.1502), Bit/dim 5.2213(best: 5.5548), Bit/dim Marginal 5.1834(best: 5.5321), Xent 1.9635, Loss 6.2030, Error 0.6876(best: 0.6744)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0410 | Time 13.6255(13.4045) | Bit/dim 5.2127(5.3096) | Xent 1.5884(1.7233) | Loss 13.6573(14.8148) | Error 0.5133(0.5814) Steps 532(550.76) | Grad Norm 27.8373(28.8970) | Total Time 0.00(0.00)
Iter 0420 | Time 13.5551(13.4416) | Bit/dim 5.1227(5.2743) | Xent 1.5858(1.7008) | Loss 13.5904(14.5372) | Error 0.5400(0.5735) Steps 556(552.91) | Grad Norm 12.2421(25.7875) | Total Time 0.00(0.00)
Iter 0430 | Time 13.5875(13.5083) | Bit/dim 5.1403(5.2461) | Xent 1.6481(1.6855) | Loss 13.6120(14.3258) | Error 0.5733(0.5708) Steps 538(551.05) | Grad Norm 25.4338(23.0509) | Total Time 0.00(0.00)
Iter 0440 | Time 12.6995(13.4591) | Bit/dim 5.1086(5.2200) | Xent 1.6265(1.6715) | Loss 13.4581(14.1458) | Error 0.5556(0.5699) Steps 544(548.76) | Grad Norm 36.3739(

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Epoch 0005 | Time 124.7888, Epoch Time 1584.3615(1774.2765), Bit/dim 4.9887(best: 5.2213), Bit/dim Marginal 4.9452(best: 5.1834), Xent 1.9163, Loss 5.9469, Error 0.6664(best: 0.6744)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0520 | Time 13.6950(13.7543) | Bit/dim 4.8915(5.0026) | Xent 1.4987(1.6091) | Loss 13.0334(13.8899) | Error 0.5200(0.5566) Steps 538(550.66) | Grad Norm 17.6945(28.5816) | Total Time 0.00(0.00)
Iter 0530 | Time 15.0144(13.8833) | Bit/dim 4.8770(4.9759) | Xent 1.4919(1.5881) | Loss 12.8670(13.6579) | Error 0.5133(0.5502) Steps 550(552.16) | Grad Norm 22.9755(26.3303) | Total Time 0.00(0.00)
Iter 0540 | Time 14.1544(13.8890) | Bit/dim 4.8370(4.9480) | Xent 1.4477(1.5742) | Loss 12.7405(13.4801) | Error 0.4911(0.5461) Steps 538(555.27) | Grad Norm 19.2269(25.8797) | Total Time 0.00(0.00)
Iter 0550 | Time 14.7280(13.9839) | Bit/dim 4.8250(4.9280) | Xent 1.5728(1.5665) | Loss 12.9125(13.3358) | Error 0.5267(0.5428) Steps 580(555.71) | Grad Norm 73.4651